In [1]:
!pip install clip_interrogator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
from clip_interrogator import Config, Interrogator
from PIL import Image

ci = Interrogator( Config( clip_model_name="ViT-L-14/openai" ) )

Loading caption model blip-large...
Loading CLIP model ViT-L-14/openai...
Loaded CLIP model and data in 14.86 seconds.


In [ ]:
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline
from clip_interrogator import Config, Interrogator
import numpy as np
import cv2


device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

In [4]:
def getRandomLine( imgSize, length : int = 40, step : int = 5, dDegree : int = 5 ) -> np.ndarray:
    #Initialize random points
    x = np.random.randint( 0, imgSize[ 1 ] )
    y = np.random.randint( 0, imgSize[ 0 ] )

    #convert degree to radiants
    dDegree = dDegree * 2 * np.math.pi / 360

    #choose a random direction in which the line should be drawn
    direction = np.random.rand( 1 ) * 2 * np.math.pi
    points = [ [ x, y ] ]

    #interate through the line and change the direction a bit
    for i in range( length ):
        direction = direction + np.random.normal( 0, 1 ) * dDegree
        x = int( x + step * np.math.cos( direction ) )
        y = int( y + step * np.math.sin( direction ) ) 
        points.append( [ x, y ] )

    points = np.array( points, dtype = np.int32 ).reshape( -1, 1, 2 )
    return points


In [33]:
import cv2
imgName = "test3"

init_image = Image.open( f"/content/gdrive/My Drive/line_{imgName}.png" ).convert( "RGB" )
init_image = init_image.resize( ( 768, 512 ) )
prompt = ci.interrogate_fast( init_image )
for i in range( 0, len( prompt ), 100 ):
  print( prompt[ i : min( i + 100, len( prompt ) ) ] )

prompt = "childish drawing style, black and white, minimalistic" + prompt 

images = pipe(prompt=prompt, image=init_image, strength=0.65, guidance_scale=7.5, negative_prompt = "different, new lines, ugly" ).images
images[0].save( f"/content/gdrive/My Drive/generated.png")


img1 = cv2.imread( f"/content/gdrive/My Drive/{imgName}.png" )
img1 = cv2.resize( img1, ( 768, 512 ) )
img1 = cv2.cvtColor( img1, cv2.COLOR_RGB2GRAY )

img2 = cv2.imread( "/content/gdrive/My Drive/generated.png" )
img2 = cv2.resize( img2, ( 768, 512 ) )
img2 = cv2.cvtColor( img2, cv2.COLOR_RGB2GRAY )


img = np.where( ( ( img1 < 200 ) | ( img2 < 200 ) ), 0, 255 )
cv2.imwrite( "/content/gdrive/My Drive/merged.png", img )

100%|██████████| 55/55 [00:00<00:00, 68.57it/s]
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['sketch art, duck shaped spaceship']


a drawing of a person holding a cell phone in their hand, highly detailed exquisite fanart, extremel
y detailed art, 8k high quality detailed art, portrait of figther jet evading, artistic drawing of a
 crow, highly detailed drawing, highly detailed art, space ship gribble, animatic, professional char
acter designer, high quality sketch art, duck shaped spaceship


  0%|          | 0/32 [00:00<?, ?it/s]

True